In [16]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score

In [17]:
# Load the datasets
X = np.load("Xtrain_Regression_Part2.npy")
X_TESTE = np.load("Xtest_Regression_Part2.npy")
Y = np.load("Ytrain_Regression_Part2.npy")

CÓDIGO PARA ENTREGAR

In [18]:
# model = linear_model.Lasso(alpha=0.002)
model = linear_model.LinearRegression()
# model = linear_model.Ridge(alpha=0.002)

score = 10
removed_value = 0

removed = 0

# train and predict
model.fit(X, Y)

Y_pred = model.predict(X)

diff = (Y - Y_pred.reshape(Y.shape[0], 1)) ** 2
avg_diff = np.average(diff)

while removed < 9:
    max_diff = max(diff)

    if max_diff > avg_diff and max_diff > removed_value / 10:
        index = np.where(diff == max_diff)

        print(f"Index {index[0][0]}: element {Y[index]} is outlier, dist = {max_diff}")

        removed_value = max_diff

        Y = np.delete(Y, index[0][0], 0)
        X = np.delete(X, index[0][0], 0)
        diff = np.delete(diff, index, 0)

        removed += 1

    else:
        break

Index 85: element [-7.15909879] is outlier, dist = [79.00975928]
Index 77: element [-0.35163306] is outlier, dist = [26.40160169]


In [19]:
# FINDING THE BEST ALPHA FOR THE LASSO
scores = {}
alphas = np.arange(0.0001, 0.2, 0.0001)
for alpha in alphas:
    scores[str(alpha)] = -1 * np.average(
        cross_val_score(
            linear_model.Lasso(alpha=alpha),
            X,
            Y,
            scoring="neg_mean_squared_error",
            cv=7,
        )
    )

best_lasso = min(list(scores.values()))
best_lasso_alpha = list(scores.keys())[list(scores.values()).index(best_lasso)]
print(f"Lasso best error {best_lasso} from alpha={ best_lasso_alpha }")

# And for Ridge
scores = {}
alphas = np.arange(0, 3, 0.01)
for alpha in alphas:
    scores[str(alpha)] = -1 * np.average(
        cross_val_score(
            linear_model.Ridge(alpha=alpha),
            X,
            Y,
            scoring="neg_mean_squared_error",
            cv=7,
        )
    )

best_ridge = min(list(scores.values()))
best_ridge_alpha = list(scores.keys())[list(scores.values()).index(best_ridge)]
print(f"Ridge best error {best_ridge} from alpha={ best_ridge_alpha }")

#linear regression
linear = -1*np.average(cross_val_score(linear_model.LinearRegression(), X, Y, scoring='neg_mean_squared_error', cv=7))
print(f"Linear best error {linear}")

scores = {"Linear": linear, "Lasso": best_lasso, "Ridge": best_ridge}
best=min( scores.values() )

Lasso best error 0.7365628619145077 from alpha=0.0212
Ridge best error 0.7463721745588999 from alpha=1.9000000000000001
Linear best error 0.7553761337006849


In [20]:
model_name = list(scores.keys())[list(scores.values()).index(best)]

print(f"Best model is {model_name}, lets predict! \N{winking face}")

if model_name == "Linear":
    model = linear_model.LinearRegression()

elif model_name == "Ridge":
    model = linear_model.Ridge(alpha=float(best_ridge_alpha))

elif model_name == "Lasso":
    model = linear_model.Lasso(alpha=float(best_lasso_alpha))

else:
    exit()


# train and predict
model.fit(X, Y)

Y_pred = model.predict(X_TESTE)

with open("predictor.npy", "wb") as f:
    np.save(f, Y_pred)


Best model is Lasso, lets predict! 😉
